In [ ]:
import sys, json

sys.path.insert(0, "../")

from lib import *

In [ ]:
user_input = {
    # "config_file": "vd_1x8x14/vd_1x8x14_config",
    "config_file": "hd_1x2x6/hd_1x2x6_config",
    "root_file": ["Marley"],
    "rewrite": True,
    "debug": True,
}
info = json.load(open("../config/" + user_input["config_file"] + ".json", "r"))
data_filter = {
    "max_energy": 20,
    "min_energy": 0,
    "pre_nhits": 3,
    "primary": True,
    "neutron": True,
}
true, data, filter_idx = compute_root_workflow(
    user_input, info, data_filter, workflow="RECONSTRUCTION", debug=user_input["debug"]
)
rprint(
    "-> Found %i electron candidates out of %i events!"
    % (len(filter_idx), data["Event"].size)
)

In [ ]:
acc = 50
fit = {"color":"grey","threshold":0.3,"trimm":10, "spec_type":"max", "print":False, "opacity": 1}
fig = make_subplots(
    rows=2, cols=3, subplot_titles=("Electron", "Gamma", "Electron+Gamma")
)

fit["func"] = "linear"
fit["spec_type"] = "top+bottom"
fig, top_bottom_popt, top_bottom_perr = get_hist2d_fit(
    data["TNuE"][filter_idx],
    data["ElectronE"][filter_idx],
    fig = fig,
    idx = (1,1),
    acc = acc,
    fit = fit,
    zoom=True
)
top_popt = top_bottom_popt[:2]
bottom_popt = top_bottom_popt[2:]
top_perr = top_bottom_perr[:2]
bottom_perr = top_bottom_perr[2:]
top_func = lambda x:  x/top_popt[0] - top_popt[1]
bottom_func = lambda x: x/bottom_popt[0] - bottom_popt[1]

# Compute reco energy
data["Discriminant"] = data["MaxAdjClEnergy"] / 8 + data["AdjClNum"] / 6
idx = data["Discriminant"] > 0.41
data["RecoEnergy"][idx] = bottom_func(data["Energy"][idx])
data["RecoEnergy"][~idx] = top_func(data["Energy"][~idx])

x, y, h = get_hist2d(
    x=data["TNuE"][filter_idx],
    y=data["GammaE"][filter_idx],
    density=True,
    acc=acc,
)
fig.add_trace(
    go.Heatmap(
        x=x,
        y=y,
        z=h.T,
        coloraxis="coloraxis",
        colorscale="Turbo",
    ), col=2, row=1
)

fit["spec_type"] = "max"
fig, popt_true, perr_true = get_hist2d_fit(
    data["TNuE"][filter_idx],
    data["VisEnergy"][filter_idx],
    fig = fig,
    idx = (1,3),
    acc = acc,
    fit = fit,
    zoom=True
)

x, y, h = get_hist2d(
    x=data["TNuE"][filter_idx],
    y=data["Energy"][filter_idx],
    density=True,
    acc=acc,
)
fig.add_trace(
    go.Heatmap(
        x=x,
        y=y,
        z=h.T,
        coloraxis="coloraxis",
        colorscale="Turbo",
    ), col=1, row=2
)

x, y, h = get_hist2d(
    x=data["TNuE"][filter_idx],
    y=data["TotalAdjClEnergy"][filter_idx],
    density=True,
    acc=acc,
)
fig.add_trace(
    go.Heatmap(
        x=x,
        y=y,
        z=h.T,
        coloraxis="coloraxis",
        colorscale="Turbo",
    ), col=2, row=2
)

fig, reco_popt, reco_perr = get_hist2d_fit(
    data["TNuE"][filter_idx],
    data["Energy"][filter_idx],
    fig,
    idx = (2,3),
    acc=acc,
    fit=fit,
    zoom=True
)

fig.update_layout(
    coloraxis=dict(colorscale="Turbo", colorbar=dict(title="Norm")),
    showlegend=False,
    yaxis1_title="True Electron Energy [MeV]",
    yaxis2_title="True Gamma Energy [MeV]",
    yaxis3_title="Combined Energy [MeV]",
    yaxis4_title="Main Cluster Energy [MeV]",
    yaxis5_title="Adj. Cluster Energy [MeV]",
    yaxis6_title="Reco Energy [MeV]",
)

fig.update_xaxes(title_text="True Neutrino Energy [MeV]")

fig = format_coustom_plotly(
    fig, fontsize=16, matches=(None, None), title="Cluster-based neutrino energy reconstruction"
)
fig.show()

In [ ]:
mid_popt = [np.mean((top_popt[0],bottom_popt[0])), np.mean((top_popt[1],bottom_popt[1]))]
top_idx = np.where(data["ElectronE"][filter_idx] > data["TNuE"][filter_idx]*mid_popt[0] + mid_popt[1])
bottom_idx = np.where(data["ElectronE"][filter_idx] < data["TNuE"][filter_idx]*mid_popt[0] + mid_popt[1])

In [ ]:
fig = make_subplots(1,2)
fig.add_trace(go.Histogram2d(coloraxis="coloraxis",x=[x for x in data["Discriminant"][filter_idx][top_idx]], y=data["TNuE"][filter_idx][top_idx],nbinsx=100,histnorm="probability"),row=1, col=1)
fig.add_trace(go.Histogram2d(coloraxis="coloraxis",x=[x for x in data["Discriminant"][filter_idx][bottom_idx]], y=data["TNuE"][filter_idx][bottom_idx],nbinsx=100,histnorm="probability"),row=1, col=2)
fig = format_coustom_plotly(fig)
fig.update_layout(coloraxis=dict(colorscale="Turbo", colorbar=dict(title="Norm")),showlegend=False, yaxis1_title="True Neutrino Energy [MeV]", xaxis1_title="Discriminant [a*MaxAdjClEnergy + b*AdjClNum]", xaxis2_title="Discriminant [a*MaxAdjClEnergy + b*AdjClNum]")
fig.show()
# fig.update_layout(barmode="overlay",bargap=0)
fig2 = go.Figure()
fig2.add_trace(go.Histogram(histnorm="probability",opacity=0.75,name="Top Calibration",x=data["Discriminant"][filter_idx][top_idx]))
fig2.add_trace(go.Histogram(histnorm="probability",opacity=0.75,name="Bottom Calibration",x=data["Discriminant"][filter_idx][bottom_idx]))
fig2 = format_coustom_plotly(fig2)
fig2.update_layout(barmode="overlay", bargap=0, title="Cluster-based neutrino energy reconstruction", xaxis_title="Discriminant [a*MaxAdjClEnergy + b*AdjClNum]", yaxis_title="Norm")
fig2.add_vline(0.41)
fig2.show()
# fig.update_yaxes(type="log")

In [ ]:
top_discriminant = data["Discriminant"][filter_idx][top_idx]
bottom_discriminant = data["Discriminant"][filter_idx][bottom_idx]
# Plot the percentage of events with discriminant > x
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=np.sort(top_discriminant), y=np.linspace(1,0,len(top_discriminant)), name="Top Calibration"))
fig3.add_trace(go.Scatter(x=np.sort(bottom_discriminant), y=np.linspace(0,1,len(bottom_discriminant)), name="Bottom Calibration"))
fig3 = format_coustom_plotly(fig3)
fig3.update_layout(title="Cluster-based neutrino energy reconstruction", xaxis_title="Discriminant [a*MaxAdjClEnergy + b*AdjClNum]", yaxis_title="Norm")
fig3.add_vline(0.41)
fig3.add_hline(0.3, line_dash="dot", line_color="grey")
fig3.show()

In [ ]:
config = user_input["config_file"].split("/")[0]
if not os.path.exists(f"../images/calibration/{config}_calibration/"):
    os.makedirs(f"../images/calibration/{config}_calibration/")
fig.write_image(
    f"../images/calibration/{config}_calibration/{config}_reco_energy.png",
    width=2400,
    height=1080,
)

rprint(
    f"-> Saved reco energy plot to ../images/calibration/{config}_calibration/{config}_reco_energy.png"
)

# Save as json file
if not os.path.exists(f"../config/{config}/{config}_calib/"):
    os.makedirs(f"../config/{config}/{config}_calib/")
with open(
    f"../config/{config}/{config}_calib/{config}_energy_calibration.json",
    "w",
) as f:
    json.dump(
        {
            "LOWER": {
                "ENERGY_AMP": bottom_popt[0],
                "INTERSECTION": bottom_popt[1]
            },
            "UPPER": {
                "ENERGY_AMP": top_popt[0],
                "INTERSECTION": top_popt[1]
            },
            "RECO": {
                "ENERGY_AMP": reco_popt[0],
                "INTERSECTION": reco_popt[1]
            },
            "TOTAL": {
                "ENERGY_AMP": reco_popt[0],
                "INTERSECTION": reco_popt[1]
            }
        },
        f
    )

rprint(f"-> Saved reco energy fit parameters to ../config/{config}/{config}_calib/{config}_energy_calibration.json")